In [1]:
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns
iris = sns.load_dataset('iris')

# convert from pandas to tensor
data = torch.Tensor(iris[iris.columns[0:4]].values).float()

# transform species to numbers
labels = torch.zeros(len(data), dtype=torch.long)
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

In [2]:
# No devset here

# how many training examples
propTraining = .8
nTraining = int(len(labels)*propTraining)
traintestBool = np.zeros(len(labels), dtype=bool)
traintestBool[range(nTraining)] = True

items4train = np.random.choice(range(len(labels)), nTraining, replace=False)
traintestBool[items4train] = True

In [3]:
#create model
model = nn.Sequential(
    nn.Linear(4, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 3)
)

lossfunction = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [4]:
numepochs = 1000
losses = torch.zeros(numepochs)
ongoingacc = []

for epochi in range(numepochs):
    yHat = model(data[traintestBool,:])
    ongoingacc.append(100*torch.mean(
        (torch.argmax(yHat,axis=1) == labels[traintestBool]).float()
    ))
    loss = lossfunction(yHat, labels[traintestBool])
    losses[epochi] = loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [5]:
predictions = model(data[traintestBool, :])
trainacc = 100*torch.mean((torch.argmax(predictions, axis=1) == labels[traintestBool]).float())

predictions = model(data[~traintestBool, :])
testacc = 100*torch.mean((torch.argmax(predictions, axis=1) == labels[~traintestBool]).float())

In [6]:
print(trainacc)
print(testacc)

tensor(98.6014)
tensor(71.4286)
